In [1]:
import sys
sys.path.append('../../ABEL')
from abel import *

In [2]:
import numpy as np
import time
import scipy.constants as SI 
import abel.utilities.plasma_physics as pp

In [3]:
# define driver
driver = SourceBasic()
driver.charge = -2.7e10 * SI.e # [C]
driver.energy = 31.25e9 # [eV]
driver.rel_energy_spread = 0.01
driver.bunch_length = 50e-6 # [m]
driver.z_offset = 580e-6 # [m]
driver.emit_nx, driver.emit_ny = 80e-6, 80e-6 # [m rad]
driver.beta_x, driver.beta_y = 5, 5 #30e-3, 30e-3 [m]
driver.num_particles = 100000
driver.jitter.x = 0#2e-7 # [m] 2e-6
driver.jitter.y = 0#2e-7 # [m] 100e-9
driver.jitter.t = 0 #10e-15 # [m]
driver.symmetrize = True

# define a stage
stage = StageHipace()
stage.driver_source = driver
stage.length = 0.11 # [m]
stage.plasma_density = 2e21 # [m^-3]
stage.nom_energy_gain = 31.9e9/(11/stage.length) # [eV]
stage.radiation_reaction = True
stage.ion_motion = False
stage.beam_ionization=False
stage.ramp_beta_mag = 5
stage.num_nodes = 4

# define witness bunch
witness = SourceBasic()
witness.charge = -1e10 * SI.e # [C]
witness.energy = 1500e9 # [eV]
witness.rel_energy_spread = 0.0
witness.bunch_length = 0e-6 # [m]
witness.z_offset = 0 # [m]
witness.emit_nx, witness.emit_ny = 160e-6, 0.56e-6 # [m rad]
witness.beta_x = stage.matched_beta_function(witness.energy)
witness.beta_y = witness.beta_x
witness.num_particles = 50000
witness.x_offset, witness.y_offset = 0, 0 # [m]
witness.jitter.x = 0
witness.symmetrize = True

# beam delivery system
bds = BeamDeliverySystemBasic()
bds.beta_x, bds.beta_y = 8e-3, 0.4e-3 # [m]
bds.bunch_length = 0.75 * bds.beta_y

linac = Linac()
linac.stage = stage
linac.source = witness
linac.num_stages = 1
shots = 1

In [4]:
linac.run(f'linac_hipace_comparisson_RR_{stage.radiation_reaction}', num_shots=shots, overwrite=True, parallel=False);

Tracking element 1 (SourceBasic, stage 0, s = 0.0 m, 1500.0 GeV, -1.60 nC, 0.0% rms, 160.2/0.6 µm-rad)


Finished HiPACE++ (job 6120339): 100%|███████| 75/75 [00:29<00:00,  2.55steps/s]
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 0:
  #000: H5F.c line 620 in H5Fopen(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3501 in H5VL_file_open(): failed to iterate over available VOL connector plugins
    major: Virtual Object Layer
    minor: Iteration failed
  #002: H5PLpath.c line 578 in H5PL__path_table_iterate(): can't iterate over plugins in plugin path '(null)'
    major: Plugin for dynamically loaded library
    minor: Iteration failed
  #003: H5PLpath.c line 620 in H5PL__path_table_iterate_process_path(): can't open directory: /usr/local/hdf5/lib/plugin
    major: Plugin for dynamically loaded library
    minor: Can't open directory or file
  #004: H5VLcallback.c line 3351 in H5VL__file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #005: H5VLnative_file.c line 97 in H5VL__native_file

Error: Read Error in backend HDF5
Object type:	File
Error type:	Inaccessible
Further description:	Failed to open HDF5 file run_data/temp/8bcb2bb7-b3c3-4d13-af00-4c86d733e418/diags/hdf5/openpmd_000075.h5

In [ ]:
Final_energies = [linac.get_beam(-1, shot = i).energy()/1e9 for i in range(shots)]
energy_standard_error = np.std(Final_energies)
mean_energy = np.mean(Final_energies)
print('RMS energy spread is ', energy_standard_error, 'Mean final energy is ', mean_energy)

gamma = (witness.energy+33e9)*SI.e/SI.m_e/SI.c**2
kp = pp.k_p(stage.plasma_density)
k_beta = kp/np.sqrt(2*gamma)

#stage.matched_beta_function(witness.energy)
sigma_sqs = np.sqrt(2*gamma)/kp/gamma*(witness.emit_nx + witness.emit_ny)

Expected_energy_loss = SI.e**2/12/np.pi/SI.epsilon_0 * (gamma*k_beta)**4*(sigma_sqs)*stage.length/SI.e/1e9
print('Expected energy loss is ', Expected_energy_loss, ' GeV')
print('Actual energy loss is ', 1535.4-1533.8, ' +- 0.06 GeV')

In [ ]:
fig = linac.stages[-1].plot_evolution()

In [ ]:
fig2 = linac.plot_evolution()